In [58]:
%%time

import numpy as np
import pandas as pd
import altair as alt
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from sklearn.linear_model import LinearRegression

# Función para resetear columnas

def reset_columns(df):
    df.columns = ['_'.join(col) for col in df.columns.values]
    return df


# Función para ponderar a total población - individuos en millones las variables de condiciones de vida y otras
def weights(dfw,weight_varlist,factor):
    for var in weight_varlist:
        dfw[var] = dfw[var] * dfw[factor]
    return dfw

df = pd.read_csv('./Files/ECV_2004_2018.csv.gz', compression = 'gzip', sep = ';')
df2 = df.copy()

# Selección de las variables necesarias para visualización y para el modelo
df2 = df2[(df2['Year'] == 2013) | (df2['Year'] == 2018) |
      (df2['Year'] == 2008) | (df2['Year'] == 2009) |
      (df2['Year'] == 2010) | (df2['Year'] == 2011) |
      (df2['Year'] == 2012) | (df2['Year'] == 2013) |
      (df2['Year'] == 2014) | (df2['Year'] == 2015) |
      (df2['Year'] == 2016) | (df2['Year'] == 2017)][['Year','HHId', 'HHHolidays','HHFood','HHReserves', 'HHComputer', 'HHCar',
                                                     'AREMonth', 'HousingCost', 'HHHeath', 'vhRentaa', 'vhPobreza', 'vhMATDEP',
                                                     'Media_renta', 'vhRentaa_zscore', 'vhRentaa_xperson', 'Habitat', 'Comunidad Autónoma_x',
                                                     'PIB_', 'Pob_', 'PIB_percapita_', 'PIB(%)_change_', 'PIB(%)_varnacional_', 'PIB_Nacional',
                                                     'Pob_Nacional', 'PIB_percapita_Nacional', 'PIB(%)_change_Nacional', 'WorStatus',
                                                     'FStudies', 'CrConditions', 'HLimitations', 'MDClothes', 'MDShoes',
                                                     'MDFriends', 'MDLeisure', 'MDSelf', 'MDInternet','HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                                     'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                                     'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                                     'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                                     'CHealth','HousingCost_LowImpactHH', 'HLimitations_NoSerLimitedG_limitado', 'vhMATDEP_vhMATDEP_No', 'MDClothes_Yes', 
                                                     'HHFood_No', 'MDShoes_Yes', 'MDShoes_No affordable', 'MDLeisure_Yes', 
                                                     'CrConditions_YChronic', 'HHWashMachine_No', 'HHWashMachine_Yes', 
                                                     'MDFriends_Yes', 'MDFriends_No affordable', 'MDInternet_Yes', 
                                                     'MDClothes_No affordable', 'HLimitations_SerLimited', 'HHCar_No', 
                                                     'HousingCost_HighImpactHH', 'MDLeisure_No affordable', 'HHHeath_No', 
                                                     'vhPobreza_vhPobreza_No', 'MDInternet_No affordable', 
                                                     'MDSelf_No affordable', 'HHComputer_No', 'HHHolidays_No', 
                                                     'HHReserves_No', 'MDSelf_Yes']].dropna()

# Estimación de la felicidad
df_model = pd.read_csv('./Files/ECV_2004&2018.csv.gz', compression = 'gzip', sep = ';')

# Meter el ID
df_model = df_model[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                      'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                      'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                      'vhRentaa', 'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                      'CHealth', 'AREMonth','LifeSatisfaction2','HousingCost_LowImpactHH', 
                                      'HLimitations_NoSerLimitedG_limitado', 'vhMATDEP_vhMATDEP_No', 'MDClothes_Yes', 
                                      'HHFood_No', 'MDShoes_Yes', 'MDShoes_No affordable', 'MDLeisure_Yes', 
                                      'CrConditions_YChronic', 'HHWashMachine_No', 'HHWashMachine_Yes', 'MDFriends_Yes', 
                                      'MDFriends_No affordable', 'MDInternet_Yes', 'MDClothes_No affordable', 
                                      'HLimitations_SerLimited', 'HHCar_No', 'HousingCost_HighImpactHH', 
                                      'MDLeisure_No affordable', 'WorStatus_EmployeeFT', 'WorStatus_Unemployed', 
                                      'HHHeath_No', 'WorStatus_Student', 'FStudies_EdSuperior', 'vhPobreza_vhPobreza_No', 
                                      'FStudies_<Primaria', 'MDInternet_No affordable', 'MDSelf_No affordable', 
                                      'HHComputer_No', 'HHHolidays_No', 'HHReserves_No', 'MDSelf_Yes']].dropna()

# Variables modelo predicciones 2008 a 2018
X = df_model[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                      'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                      'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                      'vhRentaa', 'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                      'CHealth', 'AREMonth']]
y = df_model['LifeSatisfaction2']

reg2 = LinearRegression()
reg2.fit(X2, y)
# Predicciones
df2['Predictions'] =reg2.predict(df2[['HHHolidays_Yes', 'HHFood_Yes', 'HHReserves_Yes', 
                                      'HHComputer_Yes', 'HHCar_Yes', 'HousingCost_MediumImpactHH', 
                                      'HHHeath_Yes', 'vhPobreza_vhPobreza_Yes', 'vhMATDEP_vhMATDEP_Yes',
                                      'vhRentaa', 'CrConditions_NChronic', 'HLimitations_NoLimited', 
                                      'CHealth', 'AREMonth']])

# Cambio nombre variable
df2.rename(columns = {'PIB(%)_change_Nacional': 'PIB_change_Nacional'}, inplace = True) 
# Ponderación año
df_weigths = (df2.groupby(['Year'])['Pob_Nacional'].mean() \
              / df2.groupby(['Year'])['HHId'].count()).reset_index().rename(columns = {0:'Weight'})
# Incluida ponderación por año en el df
df2 = df2.merge(df_weigths)


# Creación de quintiles con las predicciones
df2['Quintiles_bins'] = pd.qcut(df2['Predictions'],5) # Distribuir todas las observaciones predecidas en 5 quintiles

# Asignación de quintil según los rangos resultantes
df2['Quintiles'] = df2['Quintiles_bins'].apply(lambda x: '1º - Inferior' if str(x) == '(3.694, 6.298]' \
                                                        else '2º' if str(x) ==  '(6.298, 6.874]' \
                                                        else '3º' if str(x) ==  '(6.874, 7.348]' \
                                                        else '4º' if str(x) ==  '(7.348, 7.725]' \
                                                        else '5º - Superior')

# dfs para data set de visualización centrado en el contexto, datos globales de economía y 

df_context_sum = df2.groupby(['Year','Quintiles'])['Weight','PIB_percapita_Nacional',
                            'PIB_change_Nacional'].sum().reset_index()
df_context_mean = df2.groupby(['Year'])[['Predictions']].mean().reset_index()

df_context =df_context_sum.merge(df_context_mean)

# Variables en la matriz de correlaciones
var_corr = ['HHHolidays_No',
 'HHHolidays_Yes',
 'HHFood_No',
 'HHFood_Yes',
 'HHReserves_Yes',
 'HHReserves_No',
 'HHComputer_No',
 'HHComputer_Yes',
 'HHWashMachine_No',
 'HHWashMachine_Yes',
 'HHCar_No',
 'HHCar_Yes',
 'HousingCost_HighImpactHH',
 'HousingCost_LowImpactHH',
 'HousingCost_MediumImpactHH',
 'HHHeath_No',
 'HHHeath_Yes',
 'vhPobreza_vhPobreza_No',
 'vhPobreza_vhPobreza_Yes',
 'vhMATDEP_vhMATDEP_No',
 'vhMATDEP_vhMATDEP_Yes',
 'CrConditions_NChronic',
 'CrConditions_YChronic',
 'HLimitations_NoLimited',
 'HLimitations_NoSerLimitedG_limitado',
 'HLimitations_SerLimited',
 'MDClothes_No affordable',
 'MDClothes_Yes',
 'MDShoes_No affordable',
 'MDShoes_Yes',
 'MDFriends_No affordable',
 'MDFriends_Yes',
 'MDLeisure_No affordable',
 'MDLeisure_Yes',
 'MDSelf_No affordable',
 'MDSelf_Yes',
 'MDInternet_No affordable',
 'MDInternet_Yes','LifeSatisfaction2']

# Diccionario de nombres de la matriz de correlaciones 
dict_corr ={'LifeSatisfaction2':'Felicidad',
                         'HHHolidays_Yes':'Vacaciones_Sí',
                         'HHFood_Yes':'Alimentación Sí',
                         'HHReserves_Yes':'"Colchón" económico Sí',
                         'HHReserves_No':'"Colchón" económico No',
                         'HHCar_Yes':'Coche Sí',
                         'HHHeath_Yes':'Calefacción Sí',
                         'vhPobreza_vhPobreza_No':'Riesgo de pobreza No',
                         'vhMATDEP_vhMATDEP_No':'Carencia Material Severa No',
                         'MDClothes_Yes': 'Compra de ropa Sí',
                         'MDShoes_Yes':'Compra de zapatos Sí',
                         'MDFriends_Yes': 'Ocio con amigos Sí',
                         'MDLeisure_Yes': 'Ocio en general Sí',
                         'MDSelf_Yes':'Gasto en uno mismo Sí',
                         'MDInternet_Yes': 'Acceso a internet Sí',
                         'HHHolidays_No':'Vacaciones No', 
                         'HHFood_No':'Alimentación No', 
                         'HHComputer_No':'Ordenador No', 
                         'HHComputer_Yes':'Ordenador Sí',
                         'HHWashMachine_No':'Lavadora Sí',
                         'HHWashMachine_Yes':'Lavadora No', 
                         'HHCar_No': 'Coche No',
                         'HousingCost_HighImpactHH':'Alto impacto coste vivienda', 
                         'HousingCost_LowImpactHH':'Bajo impacto coste vivienda',
                         'HousingCost_MediumImpactHH':'Impacto medio coste vivienda',
                         'HHHeath_No':'Calefacción No', 
                         'vhPobreza_vhPobreza_Yes': 'Riesgo de pobreza Sí', 
                         'vhMATDEP_vhMATDEP_Yes': 'Carencia material severa',
                         'CrConditions_NChronic': 'Enfermedades Crónicas No',
                         'CrConditions_YChronic':'Enfermedades Crónicas Sí', 
                         'HLimitations_NoLimited': 'Limitaciones físicas No',
                         'HLimitations_NoSerLimitedG_limitado' : 'Limitaciones físicas leves Sí', 
                         'HLimitations_SerLimited': 'Limitaciones físicas graves Sí',
                         'MDClothes_No affordable' : 'Compra de ropa No',
                         'MDShoes_No affordable' : 'Compra de zapatos No',
                         'MDFriends_No affordable': 'Ocio con amigos No',
                         'MDLeisure_No affordable': 'Ocio en general No',
                         'MDSelf_No affordable': 'Gasto en uno mismo No',
                         'MDInternet_No affordable': 'Acceso a internet No'}


# Etiqueta codificación categoría

# Etiqueta codificación positivo / negativo

df_corr = df_model[var_corr].corr().stack().reset_index() \
              .rename(columns = {0:'correlation','level_0':'variable_1','level_1':'variable_2'})
df_corr['correlation_label'] = df_corr['correlation'].map('{:.2f}'.format)
df_corr['variable_1'] = df_corr['variable_1'].map(dict_corr)
df_corr['variable_2'] = df_corr['variable_2'].map(dict_corr)

# Guardar matriz de correlaciones
df_corr.to_csv('./Files/correlations.csv', sep = ';',index = False)

CPU times: user 7.95 s, sys: 3.42 s, total: 11.4 s
Wall time: 10 s


/home/mariacarrasco/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:108: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.


In [64]:

dictionary_map = {'LifeSatisfaction2':'Felicidad',
                         'HHHolidays_Yes':'Vacaciones_Sí',
                         'HHFood_Yes':'Alimentación Sí',
                         'HHReserves_Yes':'"Colchón" económico Sí',
                         'HHReserves_No':'"Colchón" económico No',
                         'HHCar_Yes':'Coche Sí',
                         'HHHeath_Yes':'Calefacción Sí',
                         'vhPobreza_vhPobreza_No':'Riesgo de pobreza No',
                         'vhMATDEP_vhMATDEP_No':'Carencia Material Severa No',
                         'MDClothes_Yes': 'Compra de ropa Sí',
                         'MDShoes_Yes':'Compra de zapatos Sí',
                         'MDFriends_Yes': 'Ocio con amigos Sí',
                         'MDLeisure_Yes': 'Ocio en general Sí',
                         'MDSelf_Yes':'Gasto en uno mismo Sí',
                         'MDInternet_Yes': 'Acceso a internet Sí',
                         'HHHolidays_No':'Vacaciones No', 
                         'HHFood_No':'Alimentación No', 
                         'HHComputer_No':'Ordenador No', 
                         'HHComputer_Yes':'Ordenador Sí',
                         'HHWashMachine_No':'Lavadora Sí',
                         'HHWashMachine_Yes':'Lavadora No', 
                         'HHCar_No': 'Coche No',
                         'HousingCost_HighImpactHH':'Alto impacto coste vivienda', 
                         'HousingCost_LowImpactHH':'Bajo impacto coste vivienda',
                         'HousingCost_MediumImpactHH':'Impacto medio coste vivienda',
                         'HHHeath_No':'Calefacción No', 
                         'vhPobreza_vhPobreza_Yes': 'Riesgo de pobreza Sí', 
                         'vhMATDEP_vhMATDEP_Yes': 'Carencia material severa',
                         'WorStatus_EmployeeFT': 'Trabajador a tiempo completo', 
                         'WorStatus_Student': 'Estudiante',
                         'WorStatus_Unemployed': 'Parado', 
                         'FStudies_<Primaria': 'Estudios < Primaria',
                         'FStudies_EdSuperior': 'Estudios Educación Superior',
                         'CrConditions_NChronic': 'Enfermedades Crónicas No',
                         'CrConditions_YChronic':'Enfermedades Crónicas Sí', 
                         'HLimitations_NoLimited': 'Limitaciones físicas No',
                         'HLimitations_NoSerLimitedG_limitado' : 'Limitaciones físicas leves Sí', 
                         'HLimitations_SerLimited': 'Limitaciones físicas graves Sí',
                         'MDClothes_No affordable' : 'Compra de ropa No',
                         'MDShoes_No affordable' : 'Compra de zapatos No',
                         'MDFriends_No affordable': 'Ocio con amigos No',
                         'MDLeisure_No affordable': 'Ocio en general No',
                         'MDSelf_No affordable': 'Gasto en uno mismo No',
                         'MDInternet_No affordable': 'Acceso a internet No'}

In [ ]:
df_model = df_model.merge(df_weigths) # Aplicación de los pesos de ponderación de cara a usarlo en alguna visualización
df_model.to_csv('./Files/Modelo.csv', sep = ';', index = False)